# Chat Completion API

In [ ]:
!pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google colab - secret 탭에 'OPENAI_API_KEY' 등록

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')


In [2]:

from openai import OpenAI
client = OpenAI(
    api_key=OPENAI_API_KEY,
)


In [ ]:
response = client.chat.completions.create(
  model="gpt-4.1",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    },
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
response.choices[0].message.content

'안녕, 차은우! 만나서 정말 반가워~ 😊  \n오늘은 어떤 이야기를 나누고 싶어? 궁금한 거나 도움이 필요한 게 있으면 언제든 말해줘!'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시 사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력 지시

## 기사 제목 교정
- 기사들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스 AFP 속보 시스템에서 도입되어 사

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환.'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어 교정 작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있을 경우 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사 제목을 읽고 주요 내용을 이해하세요.
2. 제목이 전달하고자 하는 메시지를 명확히 반영했는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결히 수정하세요.

### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목 : [기본 기사 제목]
- 교정 제목 : [교정 기사 제목]

### Examples ###
- 원래 제목 : "어제 서울에서 큰 불이 나 수백 명이 대피했다."
- 교정 제목 : "서울 대형 화제, 수백 명 대피"

- 원래 제목: "오늘 부산 앞바다에서 큰 사고가 발생해 수십 명이 구조됐다."
- 교정 제목: "부산 해상 사고, 수십 명 구조"

### Extra Instructions ###
- 제목이 너무 길면, 간결히 줄이되, 핵심 메시지를 잃어서는 안 됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사 제목을 간결하게 수정해 주세요.
{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4.1",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    },
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목 : 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환.
- 교정 제목 : 힘든 작업에 끼니 거르는 노동자들의 고충


In [ ]:
# 함수 리팩토링
def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
  # 역할/페르소나/지시사항
  system_message = """
  기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어 교정 작업을 수행해 주세요.

  - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
  - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
  - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나 중립적인 어조로 수정하세요.
  - 비속어가 포함되어 있을 경우 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

  ### Steps ###
  1. 기사 제목을 읽고 주요 내용을 이해하세요.
  2. 제목이 전달하고자 하는 메시지를 명확히 반영했는지 검토하세요.
  3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
  4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결히 수정하세요.

  ### Output Format ###
  기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

  - 원래 제목 : [기본 기사 제목]
  - 교정 제목 : [교정 기사 제목]

  ### Examples ###
  - 원래 제목 : "어제 서울에서 큰 불이 나 수백 명이 대피했다."
  - 교정 제목 : "서울 대형 화제, 수백 명 대피"

  - 원래 제목: "오늘 부산 앞바다에서 큰 사고가 발생해 수십 명이 구조됐다."
  - 교정 제목: "부산 해상 사고, 수십 명 구조"

  ### Extra Instructions ###
  - 제목이 너무 길면, 간결히 줄이되, 핵심 메시지를 잃어서는 안 됩니다.
  - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
  - 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.
  """

  user_message = f"""
  다음 기사 제목을 간결하게 수정해 주세요.
  {title_before}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      },
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

title_before = '주말 미친듯한 폭우 예상, 모두들 무사하시길'
output = correct_news_title(title_before)
print(output)


- 원래 제목 : "주말 미친듯한 폭우 예상, 모두들 무사하시길"
- 교정 제목 : "주말 폭우 예보, 안전 유의하세요"


## 연애 코치 ReAct

In [4]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
  # 역할/페르소나/지시사항
  system_message = """
  <<system instruction>>
  어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로서 사용자의 고민을 해결해 주세요.

  <<output format>>
  1. 상황 분석:
  2. 행동 계획:
  3. 실행:
  """

  user_message = f"""
  사용자 현재 상황:
  {prompt}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      },
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [6]:
print(dating_coach(''))

1. 상황 분석: 
   100일 기념일은 특별한 의미를 가진 날로, 서로의 관계를 되돌아보고 더욱 깊은 애정을 나눌 기회입니다. 여자친구가 어떤 것을 좋아하는지, 그리고 두 분의 관계에서 어떤 순간들이 특히 기억에 남는지를 고려하면 더욱 의미 있는 선물을 선택할 수 있습니다. 감성을 자극하면서도 실용적인 아이템이 좋습니다.

2. 행동 계획: 
   - **여자친구의 선호 파악**: 그녀가 좋아하는 취미나 관심사, 색상 등을 미리 조사해보세요.
   - **추억의 심화**: 두 분이 함께한 특별한 순간들을 이야기할 수 있는 기회를 만들어 보세요. 마치 연애의 연대기를 돌아보는 듯한 감성을 줄 수 있습니다.
   - **선물 아이디어 준비**: 아래에 몇 가지 선물 아이디어를 작성해보았습니다.
     - *맞춤형 주얼리*: 그녀의 이름이나 이니셜이 새겨진 목걸이나 팔찌.
     - *기념 사진 앨범*: 함께한 사진들을 모아 특별한 메시지를 담아 앨범으로 제작.
     - *데이트 카드*: 100일 기념일을 기념하는 특별한 데이트를 계획하여 카드로 제공.
     - *화환이나 꽃다발*: 그녀가 좋아하는 꽃으로 특별한 부케를 만들어 주는 것도 좋습니다.
     - *우정과 사랑의 편지*: 자신의 진솔한 감정을 담아 쓴 편지를 함께 선물.

3. 실행: 
   - 선물 아이디어 중 하나를 선택한 뒤, 시간을 넉넉히 두고 준비하세요. 
   - 주말에 그녀와의 데이트를 계획하고, 선물을 주는 순간을 특별하게 만들어 보세요. 
   - 특별한 장소에서 혹은 로맨틱한 저녁식사 후, 선물을 서프라이즈로 건네며 함께했던 추억을 이야기하면 더욱 감동적인 순간이 될 것입니다.
   - 중요한 것은 선물보다 당신의 정성과 진심이 그녀에게 전달되는 것이니, 그것을 잊지 마세요.


## 냉털 마스터 ReAct
- 사용자는 냉장고에 남아있는 음식 재료를 알려주면 LLM은 이를 바탕으로 어떤 음식을 만들지 조언
- Reasoning/Action 기법을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요.

In [15]:
def fridge_raid(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
  system_message = """
  <<system instruction>>
  너는 이제부터 여러 가지 재료를 바탕으로 많은 요리를 만들 수 있는 요리사이며 그런 지식을 바탕으로 사람들이 재료를
  알려주면 그 재료를 바탕으로 만들 수 있는 적절한 요리와 그 요리법, 그리고 시간이 최대 20분 정도의
  짧게 끝마칠 수 있는 요리를 알려줘야 해. 근데 입력받은 재료들을 전부 쓸 필요는 없이 그 재료들 중
  몇 개를 골라서 사용해도 돼.

  <<과정>>
  1. 재료를 분석하여 어떤 요리를 선택하는 게 좋은지 선택
  2. 선택한 요리를 조리하는 방법을 출력

  <<output format>>
  재료:
  요리명:
  요리법:

  <<예시>>
  재료:밥, 참기름, 햄, 김치, 달걀, 김가루, 식용류, 소금, 설탕, 대파
  요리명:김치볶음밥
  요리법:
  재료 준비:
  1.신김치는 속을 가볍게 털어내고 가위를 이용해 잘게 썰어줍니다. 김칫 국물도 2~3큰술 따로 준비합니다.
  2.돼지고기와 대파는 잘게 썰어 준비합니다.
  3.돼지고기를 사용할 경우, 소금과 후추로 가볍게 밑간을 해두면 좋습니다.
  파기름 내기:
  1.중불로 달군 팬에 식용유를 넉넉히 두르고 송송 썬 대파를 넣어 볶습니다.
  2.파의 향이 기름에 배어 나올 때까지 충분히 볶아 '파기름'을 만들어 줍니다. 이것이 볶음밥 풍미의 기초가 됩니다.
  고기와 김치 볶기:
  1.파기름에 밑간 한 돼지고기를 넣고 겉이 노릇해질 때까지 볶아줍니다.
  2.고기가 익으면 잘게 썬 김치와 설탕 반 큰술을 넣고 함께 볶습니다. 설탕은 김치의 신맛을 부드럽게 잡아주고 감칠맛을 더합니다.
  3.김치가 나른해지고 투명한 빛을 띨 때까지 중불에서 충분히 볶아주는 것이 중요합니다.
  불맛 입히고 밥 볶기:
  1.볶은 재료를 팬 한쪽으로 밀어놓고, 빈 공간에 간장 1큰술을 부어 살짝 눌어붙게 끓입니다. '지글지글' 소리가 나면서 간장이 끓으면 재료와 함께 빠르게 섞어 불맛을 입혀줍니다.
  2.불을 잠시 끄거나 약불로 줄인 후, 준비된 찬밥을 넣습니다. 주걱을 세워 밥알을 자르듯이 섞어주면 양념이 고루 배고 밥이 떡지는 것을 막을 수 있습니다.
  3.밥알이 고슬고슬하게 풀리고 양념과 잘 섞였다면, 김칫 국물을 2~3큰술 넣어 맛의 깊이를 더합니다.
  마무리:
  1.불을 끈 뒤 참기름과 통깨를 뿌려 가볍게 섞어줍니다.
  2.완성된 김치볶음밥을 그릇에 담고, 취향에 따라 반숙 달걀 프라이와 김가루를 올려 마무리합니다.

  """

  user_message = f"""
  재료:
  {prompt}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      },
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [22]:
from IPython.display import Markdown

display(Markdown(fridge_raid('어묵, 소금, 두부, 된장, 젓갈, 양파, 생면')))

재료: 어묵, 두부, 된장, 양파 
요리명: 어묵 두부 찌개
요리법:
재료 준비:
1. 어묵은 먹기 좋은 크기로 자릅니다.
2. 두부는 1cm 두께로 썰어줍니다.
3. 양파는 채썰어 준비합니다.

조리 방법:
1. 냄비에 물을 넣고 끓이기 시작합니다. 약 500ml 정도의 물이 적당합니다.
2. 물이 끓으면 된장을 풀어 넣고 잘 섞어줍니다.
3. 된장이 잘 풀어지면 어묵과 양파를 넣고 중불에서 5분 정도 끓입니다.
4. 어묵이 익어갈 즈음 두부를 넣고 3-4분 정도 더 끓입니다.
5. 간을 보고 소금으로 맛을 조절합니다.

마무리:
1. 완성된 어묵 두부 찌개를 그릇에 담고 원하시면 고추가루나 대파를 뿌려 곁들여 주세요.
2. 따뜻한 밥과 함께 제공하면 좋습니다. 

총 조리 시간: 약 20분.

## 면접 질문 생성 JSON 출력

In [24]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
  system_message = """
  당신은 머신러닝/딥러닝/LLM/AI 서비스 개발의 전문가로서, 해당 분야의 제일가는 면접관입니다.
  매번 그룹사의 인재를 발굴하기 위해 면접 질문/모범 답안을 작성하고 있습니다.

  <<지시 사항>>
  - 사용자가 제출한 채용공고 내용을 바탕으로 예상면접 질문과 답변을 작성해주세요.
  - 하드 스킬과 소프트 스킬 두 개의 섹션으로 나눠서 작성해주세요.
  - 각 스킬별로 질문/답변을 3개씩 만들어주세요.

  <<출력 형식>>
  출력은 반드시 json 형식으로만 반환되어야 합니다.
  {{
    "hard_skill":[
      {{
        "question": "",
        "answer": ""
      }},
      ...
    ],
    "soft_skill":[
      {{
        "question": "",
        "answer": ""
      }}
    ],
  }}

  """

  user_message = f"""
  채용공고:
  {job_posting}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      },
    ],
    response_format={
      "type": "json_object"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [29]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""

output = job_interview(job_posting)
print(type(output))
print(output)


<class 'str'>
{
  "hard_skill":[
    {
      "question": "Python에서 머신러닝 모델을 배포하기 위해 어떤 패키지나 라이브러리를 사용할 수 있나요?",
      "answer": "머신러닝 모델을 배포하기 위해 Flask 또는 FastAPI와 같은 웹 프레임워크를 사용할 수 있습니다. 또한, TensorFlow Serving이나 MLflow와 같은 전용 서비스도 활용할 수 있습니다. 이들을 통해 모델을 REST API로 제공하게 되며, 클라이언트가 이를 호출하여 예측을 요청할 수 있습니다."
    },
    {
      "question": "Kubernetes에서 컨테이너를 어떻게 관리하고, 오케스트레이션할 수 있나요?",
      "answer": "Kubernetes는 클러스터 내에서 여러 컨테이너화된 애플리케이션을 관리합니다. pod, service, deployment 등의 리소스를 정의하여 필요시 스케일링하거나 롤아웃을 수행합니다. Helm을 이용한 패키지 관리 및 CI/CD 파이프라인을 설정하여 지속적 배포를 자동화할 수 있습니다."
    },
    {
      "question": "Jenkins를 사용하여 CI/CD 파이프라인을 구축하는 절차는 어떻게 되나요?",
      "answer": "Jenkins를 사용하여 CI/CD 파이프라인을 구축할 때, 먼저 Jenkinsfile을 생성하여 빌드, 테스트, 배포 단계를 정의합니다. Jenkins 서버에 해당 파일을 구성하고, GitHub와 연동하여 코드 푸시 시 자동으로 파이프라인이 트리거되도록 설정합니다. 필요시 슬랙과 같은 메신저와 연동해 알림을 설정할 수 있습니다."
    }
  ],
  "soft_skill":[
    {
      "question": "팀원과의 협업할 때 가장 중요하게 생각하는 점은 무엇인가요?",
      "answer": "팀원과의 협업에서 가장 중요하게 생각하는 점은 소통입니다. 각자의 역할과 책

In [28]:
import json

# 응답 데이터가 json 형식이 아니라면 오류 발생!
data = json.loads(output)
print(type(data))
print(data)

<class 'dict'>
{'hard_skill': [{'question': 'AI 모델을 배포하기 위한 인프라 구축 시 가장 중요하게 고려해야 할 요소는 무엇인가요?', 'answer': 'AI 모델 배포를 위한 인프라 구축 시 가장 중요하게 고려해야 할 요소는 확장성입니다. 인프라는 변화하는 데이터와 트래픽에 따라 확장 가능해야 하며, 안정성과 성능을 유지해야 합니다. 또한, 모니터링 및 로깅 시스템을 통해 인프라의 상태를 실시간으로 파악할 수 있도록 설계해야 합니다.'}, {'question': 'Kubernetes를 사용한 컨테이너 오케스트레이션의 장점은 무엇인가요?', 'answer': 'Kubernetes는 고가용성과 자동 복구, 자원 관리의 효율성을 제공하여 컨테이너 기반 환경의 구축과 운영을 간소화합니다. 또한, 쉽게 스케일 아웃/스케일 인이 가능하여, 수요 변동에 적절히 대응할 수 있습니다. CI/CD 통합이 용이하여 배포 프로세스를 자동화할 수 있는 장점도 있습니다.'}, {'question': 'DevOps 환경에서 Jenkins와 GitLab CI/CD의 차이점은 무엇인가요?', 'answer': 'Jenkins는 오픈 소스 자동화 서버로, 다양한 플러그인을 지원하여 유연하게 CI/CD 파이프라인을 구축할 수 있는 반면, GitLab CI/CD는 GitLab에 내장된 CI/CD 기능으로, GitOps 환경을 기반으로 통합 관리할 수 있습니다. Jenkins는 별도의 서버에서 작동하지만, GitLab CI/CD는 리포지토리와 함께 관리되기 때문에 더 쉽게 설정하고 협업할 수 있습니다.'}], 'soft_skill': [{'question': 'AI 인프라 팀에서의 협업을 위해 어떤 커뮤니케이션 방식을 선호하시나요?', 'answer': '저는 정기적인 스크럼과 같은 팀 미팅을 통해 팀원들과의 의견 교환을 선호합니다. 또한, 슬랙이나 팀즈와 같은 협업 툴을 활용하여 비동기적으로도 소통할 수 있도록 합니다. 이러한 방법은 신속한 피드백과 문제